In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import os
import scipy.signal
import fasttext

from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score,accuracy_score
from numba import jit, prange
from tqdm import tqdm
from vmdpy import VMD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
os.chdir('../')
os.chdir('data/')

In [2]:
def format_text(df,col):
      comp_df = df.copy()
      # remove all the punctuation
      comp_df[col] = comp_df[col].str.replace(r'(@\w*)','')

      #Remove all non-character
      comp_df[col] = comp_df[col].str.replace(r"[^a-zA-Z ]","")

      # Remove extra space
      comp_df[col] = comp_df[col].str.strip()

      # Change to lowercase
      comp_df[col] = comp_df[col].str.lower()
      return comp_df

def energy(u):
# Estimate PSD `S_xx_welch` at discrete frequencies `f_welch`
    f_welch, S_xx_welch = scipy.signal.welch(u)
    # Integrate PSD over spectral bandwidth
    # to obtain signal power `P_welch`
    df_welch = f_welch[1] - f_welch[0]
    return np.sum(S_xx_welch) * df_welch  

def maxvdm(f):
    alpha = 2     
    tau = 0            
    K = 2       
    DC = 0             
    init = 1           
    tol = 1e-9
    u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol) 
    energy_array=[]
    for i in u:
        energy_array.append(energy(i))
    ind = np.argmax(energy_array)
    return u[ind]

In [3]:
def tune(f,alpha,K):   
    tau = 0                 
    DC = 0             
    init = 1           
    tol = 1e-9
    u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol) 
    energy_array=[]
    for i in u:
        energy_array.append(energy(i))
    ind = np.argmax(energy_array)
    return u[ind]

@jit(parallel=True,forceobj=True)
def tuning(features,y):
    f_dict={}
    acc_dict={}
    for k in prange(2,5):
        for al in tqdm(prange(2,6)):
            X = []
            for i in prange(len(features)):
                X.append(tune(features[i],al,k))
                
            X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25)
            
            lr = LogisticRegression()
            lr.fit(X_train,y_train)
            y_pred = lr.predict(X_test)
            
            f = f1_score(y_test, y_pred)
            a = accuracy_score(y_test, y_pred)
            f_dict['K : '+str(k)+' Alpha : '+str(al)] = f
            acc_dict['K : '+str(k)+' Alpha : '+str(al)] = a
    
    return f_dict,acc_dict

In [4]:
tweets = pd.read_csv('train.csv')

In [5]:
train = format_text(tweets,'text')
X = train['text'].tolist()
Y = train['target']

In [6]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(X).toarray()

In [7]:
def vm(features):
    X = np.zeros((features.shape[0],features.shape[1]))
    for i in tqdm(prange(len(features))):
        X[i]=maxvdm(features[i])
    return X

X_vmd = vm(features)

  0%|          | 19/7613 [00:02<16:10,  7.82it/s]/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/vmdpy/vmdpy.py:105: RuntimeWarning: invalid value encountered in double_scalars
  omega_plus[n+1,k] = np.dot(freqs[T//2:T],(abs(u_hat_plus[n+1, T//2:T, k])**2))/np.sum(abs(u_hat_plus[n+1,T//2:T,k])**2)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/vmdpy/vmdpy.py:114: RuntimeWarning: invalid value encountered in double_scalars
  omega_plus[n+1,k] = np.dot(freqs[T//2:T],(abs(u_hat_plus[n+1, T//2:T, k])**2))/np.sum(abs(u_hat_plus[n+1,T//2:T,k])**2)
100%|██████████| 7613/7613 [15:40<00:00,  8.10it/s]


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_vmd, Y, test_size=0.25)

In [42]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.83      1102
           1       0.84      0.58      0.68       802

    accuracy                           0.78      1904
   macro avg       0.79      0.75      0.75      1904
weighted avg       0.79      0.78      0.77      1904



In [17]:
ft = fasttext.load_model('cc.en.300.bin')
size = ft.get_dimension()
corpus = []
for i in X:
    corpus.append(word_tokenize(i))
X_fast = np.zeros((len(corpus),size)) #Initializing the X matrix with zeros
for i in range(len(corpus)):
    emb = np.zeros((1,size))
    for w in corpus[i]:
        emb = emb +  ft.get_word_vector(w)
    X_fast[i] = emb

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_fast, Y, test_size=0.25)
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81      1098
           1       0.75      0.71      0.73       806

    accuracy                           0.78      1904
   macro avg       0.78      0.77      0.77      1904
weighted avg       0.78      0.78      0.78      1904



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
